<a href="https://colab.research.google.com/github/marceloflores-soa/EA3-Colab/blob/main/SOA_Ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descripción Ejecicio 1
El ejercicio plantea la implementación de 2 algoritmos: Producto Escalar y Ordenamiento QuickSort. Esto fue realizado para identificar velocidades y beneficios en una operación aritmetica con el producto escalar y un ordenamiento de vector en recursividad.
Ambos algoritmos plantean dos versiones, la primera de ellas deberá ejecutarse en el entorno CPU con el lenguaje Python y la segunda en el entorno GPU utilizando el lenguaje Cuda junto con la librería pycuda.

La finalidad del ejercicio es mostrar la ejecución de un producto escalar para CPU y GPU.

**Producto Escalar**

El producto escalar de dos vectores  𝑎→ y 𝑏→ devuelve un escalar que se obtiene como la suma de las multiplicaciones una a una de las componentes cartesianas de los 2 vectores 𝑎→ y 𝑏→.

**QuickSort**

Éste algoritmo se basa en el principio de divide y conquistarás. Resulta más fácil ordenar listas pequeñas que una grande, con lo cual irá descomponiendo la lista en dos partes y ordenando esas partes. Para ésto utiliza la recursividad.


Se medirán los tiempos que demora la ejecución en CPU y GPU, y se hará foco en la utilización de paralelimos a partir de GPU.

Para entender mejor lo mencionado, ejecutaremos las pruebas y veremos entonces las conclusiones.

# Instalación de PyCuda

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.3MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620900 sha256=f7303e8dca6a947334e34e7cb6ca9fb7391db8e7ce37db7bbd7c5787fb922b8e
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=b7f89100cf15900766d59454179703bc20ec79e31fde049451e24fca4ab11264
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


# PRODUCTO ESCALAR

## Desarrollo - CPU

In [ ]:
# --------------------------------------------
#@title Ingrese el tamaño del vector { vertical-output: true }

N = 100000#@param {type: "number"}
if (N <= 0):
  raise Exception("Solo se aceptan números positivos") 
  
# ---Importamos las librerías de CUDA
import random
import numpy as np 
from datetime import datetime
tiempo_total = datetime.now()

# ---Definición de función para producto escalar 
def dot(K, L):
   if len(K) != len(L):
      return 0
   return sum(i[0] * i[1] for i in zip(K, L))


# ---Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# Creamos valores entre 0 y 1 para producto escalar
h_a = np.random.randn(1,N)
h_b = np.random.randn(1, N)

print("Vector A: ", np.array2string(h_a, precision=3, separator=',', suppress_small=True))
print("")
print("Vector B: ", np.array2string(h_b, precision=3, separator=',', suppress_small=True))

# Calculamos el producto escalar
h_c = dot(h_a, h_b)


# Mostramos los resultados por pantalla
print("")
print("Producto Escalar: ", np.array2string(h_c, precision=3, separator=',', suppress_small=True)," = ", np.sum(h_c))
print("")
print("---TIEMPOS DE CPU RESULTANTES---")
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )


Vector A:  [[-1.672, 1.92 , 2.629,...,-0.1  ,-0.5  , 1.508]]

Vector B:  [[-0.367, 0.533,-0.741,...,-0.318,-0.213,-1.326]]

Producto Escalar:  [ 0.614, 1.024,-1.947,..., 0.032, 0.106,-1.999]  =  -240.9154600378153

---TIEMPOS DE CPU RESULTANTES---
Tiempo CPU:  14.59 [ms]


## Desarrollo GPU

* Para poder ejecutar el siguiente desarrollo, por favor, cambie el **tipo de entorno de ejecución** a GPU. Ingresando en "Herramientas" -> "Tipo de entorno de ejecución".

In [ ]:
# --------------------------------------------
#@title Ingrese el tamaño del vector { vertical-output: true }
# Ingresamos el tamaño de los vectores
N = 100000#@param {type: "number"}

if (N <= 0):
  raise Exception("Solo se aceptan números positivos") 
# --------------------------------------------
# ---Importamos las librerías de CUDA
import numpy as np
from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# ---Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# ---Configuración de ejecución de hilos
def iDivUp(a, b):
    return a // b + 1

# ---Main

# Comenzamos a contar el tiempo de procesamiento
tiempo_total = datetime.now()

dim_hilo = 256

# Creamos valores entre 0 y 1 para producto escalar
h_a = np.random.randn(1,N)
h_b = np.random.randn(1, N)

print("Vector A: ", np.array2string(h_a, precision=3, separator=',', suppress_small=True))
print("")
print("Vector B: ", np.array2string(h_b, precision=3, separator=',', suppress_small=True))

# Defino los 3 vectores para el producto escalar
h_a = h_a.astype(np.float32)
h_b = h_b.astype(np.float32)
h_c = np.empty_like(h_a)

# Defino la función kernel que ejecutará en GPU
module = SourceModule("""
__global__ void dotProduct(float * __restrict__ d_c, const float * __restrict__ d_a, 
                                                    const float * __restrict__ d_b,
                                                    const int N)
{
  const int tid = threadIdx.x + blockIdx.x * blockDim.x;
  if (tid >= N) return;
  d_c[tid] = d_a[tid] * d_b[tid];
}
""")

# Defino la función para el producto escalar
kernel = module.get_function("dotProduct")

# Ejecuta el kernel
blockDim = (dim_hilo, 1, 1)
gridDim = (iDivUp(N, dim_hilo), 1, 1)

tiempo_gpu = datetime.now()

kernel(cuda.Out(h_c), cuda.In(h_a), cuda.In(h_b), np.int32(N), block = blockDim, grid = gridDim)

tiempo_gpu = datetime.now() - tiempo_gpu



tiempo_total = datetime.now() - tiempo_total

# Mostramos los resultados por pantalla
print("")
print("Producto Escalar: ", np.array2string(h_c, precision=3, separator=',', suppress_small=True)," = ", np.sum(h_c))
print("---TIEMPOS DE CPU y GPU RESULTANTES---")
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )



ModuleNotFoundError: ignored

## Tablas de pasos

### *CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  np.random.randn(1,N) | Inicializa los vectores **h_a y h_b** con cantidad_N de números random entre el 0 y el 1.
CPU      |  dot(K, L)      | Realiza el producto cartesiano.
CPU      |  tiempo_en_ms      | Transforma el tiempo en milisegundos.
CPU      |  print()               | Informo los resultados.

### *GPU

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  np.random.randn(1,N) | Inicializa los vectores h_a y h_b con cantidad_N de números random entre el 0 y el 1.
CPU      |  np.astype(float32)            | Defino los valores dentro de los vectores como float32.
CPU      |  np.empty_like( **h_a** ) | Genera un array vacio del mismo tipo que **h_a** y se lo asigna a **h_c**.
**GPU**  |  pycuda.driver.in()      | Indica que el array debe copiarse en el dispositivo de cómputo antes de invocar el kernel.
**GPU**  |  pycuda.driver.out()    | Indica que el array debe copiarse del dispositivo de cómputo después de invocar el kernel.
CPU      |  SourceModule()        | Define el código del kernel. 
CPU      |  module.get_function() | Genera la función del kernel GPU.
CPU      |  iDivUp         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  tiempo_en_ms()               | Transforma el tiempo en milisegundos.
CPU      |  print()               | Informo los resultados.

## Conclución sobre Producto Escalar

Al ejecutar el algoritmo en CPU y GPU, ingresando un valor alto para la variable N, la implementación realizada con el procesador GPU da una clara ventaja utilizando su paralelismo y logra tiempos mucho más cortos para realizar el producto escalar.

Los algoritmos en GPU son eficientes cuando el problema presenta una alta intensidad aritmética, que puede definirse como el número de operaciones aritméticas por dato leído / escrito. En conclusión, la ejecución en GPU demuestra un tiempo inferior en realizar las operaciones que procesando en CPU.

Con el fin de buscar un algoritmo más complejo que el producto escalar, implementé un ordenamiento del tipo QuickSort el cual utiliza un método recursivo para ordenar un vector. 


# QUICKSORT

## Desarrollo - CPU

In [ ]:
# --------------------------------------------
#@title Ingrese el tamaño del vector { vertical-output: true }

# ---Importamos las librerías de CUDA
import random
import numpy as np
from datetime import datetime


tamaño_listado =   50#@param {type: "number"}

if (tamaño_listado <= 0):
  raise Exception("Solo se aceptan números positivos") 


tiempo_total = datetime.now()

# ---Función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# ---Defino valores del listado
arr = [random.randint(1,10) for _ in range(tamaño_listado)]

print( "Listado original: ", arr)

# ---Función QuickSort
def quicksort(arr):
          if len(arr) <= 1:
             return arr
          pivot = arr[len(arr)//2]
          left = [x for x in arr if x < pivot]
          middle = [x for x in arr if x == pivot]
          right = [x for x in arr if x > pivot]
          return quicksort(left) + middle + quicksort(right)     

# ---MAIN
tiempo_ejecucion = datetime.now()             
# Ejecuto ordenamiento
resultado = quicksort(arr) 
tiempo_ejecucion = datetime.now() - tiempo_total
# Mostramos el listado ordenado
print( "Listado ordenado: ", resultado )
print( "------------------------------------")

# Mostramos tiempos de CPU
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo Ejecucion de Ordenamiento: ", tiempo_en_ms( tiempo_ejecucion ), "[ms]" )

## Desarrollo - GPU

* Para poder ejecutar el siguiente desarrollo, por favor, cambie el **tipo de entorno de ejecución** a GPU. Ingresando en "Herramientas" -> "Tipo de entorno de ejecución".

In [ ]:
# --------------------------------------------
#@title Ingrese el tamaño del vector { vertical-output: true }

# ---Importamos las librerías de CUDA
import random
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda import gpuarray, compiler
from pycuda.compiler import SourceModule
import time
from datetime import datetime
import numpy as np

tamaño_listado =   50#@param {type: "number"}

if (tamaño_listado <= 0):
  raise Exception("Solo se aceptan números positivos") 

tiempo_total = datetime.now()
tiempo_gpu = datetime.now()
tiempo_gpu_final = 0


# ---Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# ---Definición de función que realiza el ordenamiennto QuickSort 
def quicksort(arr):

  global tiempo_gpu
  global tiempo_gpu_final

  if len(arr) <= 1:
    return arr
 
  else:

    pivot = arr.pop()
    pivot = np.int32(pivot)

    kernel_function = """
    #include <stdio.h>
                    
    __global__ void sort(int *arr, int *arr_aux1, int *arr_aux2, int *l1_size, int *l2_size, int pivot)
    {
      int index = blockIdx.x * blockDim.x + threadIdx.x;
      int stride = blockDim.x * gridDim.x;            
      for (int i = index; i < %(ARRAY_SIZE)s; i+= stride){
        if (arr[i] < pivot)
        {
          arr_aux1[atomicAdd(l1_size, 1)] = arr[i];
        }else{
              arr_aux2[atomicAdd(l2_size, 1)] = arr[i];
              }
        __syncthreads();
      }
    }
    """
    tamaño_listado = len(arr)

    # CPU - reservo la memoria GPU.
    arr = np.asarray(arr)
    arr = arr.astype(np.int32)
    lista_gpu = cuda.mem_alloc(arr.nbytes)
    lista_aux1_gpu = cuda.mem_alloc(arr.nbytes)
    lista_aux2_gpu = cuda.mem_alloc(arr.nbytes)
    lista_aux1_tamaño   = cuda.mem_alloc(4)
    lista_aux2_tamaño   = cuda.mem_alloc(4)
    lista_aux1_sh = np.zeros(1, dtype = np.int32)
    lista_aux2_sh = np.zeros(1, dtype = np.int32)

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod(lista_gpu, arr)
    cuda.memcpy_htod(lista_aux1_tamaño, lista_aux1_sh)
    cuda.memcpy_htod(lista_aux2_tamaño, lista_aux2_sh)

    # GPU - Ejecuta el kernel
    dim_hilo = 256
    dim_bloque = np.int( (tamaño_listado+dim_hilo-1) / dim_hilo )
    
    kernel_code = kernel_function % {'ARRAY_SIZE': tamaño_listado}
    module = compiler.SourceModule(kernel_code)
    kernel = module.get_function("sort")
    tiempo_gpu = datetime.now()
    kernel(lista_gpu, lista_aux1_gpu, lista_aux2_gpu, lista_aux1_tamaño, lista_aux2_tamaño, pivot, block=(dim_hilo, 1, 1), grid=(dim_bloque, 1))
    tiempo_gpu = datetime.now() - tiempo_gpu 

    tiempo_gpu_final = tiempo_gpu_final + tiempo_gpu.total_seconds()

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh(lista_aux1_sh, lista_aux1_tamaño)
    cuda.memcpy_dtoh(lista_aux2_sh, lista_aux2_tamaño)
    arr_aux1 = np.zeros(lista_aux1_sh, dtype=np.int32)
    arr_aux2 = np.zeros(lista_aux2_sh, dtype=np.int32)
    cuda.memcpy_dtoh(arr_aux1, lista_aux1_gpu)
    cuda.memcpy_dtoh(arr_aux2, lista_aux2_gpu)
    arr_aux1 = arr_aux1.tolist()
    arr_aux2 = arr_aux2.tolist()

  return quicksort(arr_aux1) + [pivot] + quicksort(arr_aux2)

# ---MAIN
# Defino valores del listado
arr = [random.randint(1,10) for _ in range(tamaño_listado)]
print( "Listado original: ", arr)
# Llamamos a la funcion ordenamiento 
resultado = quicksort(arr)
print( "Listado por quicksort: ", resultado)
print( "------------------------------------")
# Informamos los tiempos
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_gpu_final , "[ms]" )




## Tablas de pasos

### CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  random.randint(1,10) for _ in range(cantidad_N) | Inicializa el vector **arr** con cantidad_N de números random entre el 1 y el 10.
CPU      |  quicksort(arr)      | Ordena el vector con recursividad.
CPU      |  print()               | Informo los resultados.

### GPU

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  random.randint(1,10) for _ in range(cantidad_N) | Inicializa el vector **x_cpu** con cantidad_N de números random entre el 1 y el 10.
CPU      |  quicksort(arr)            | Función que ordena el vector.
CPU      |  np.array()            | Defino los valores dentro del array **x_cpu** como int32.
CPU      |  np.empty_like( **x_cpu** ) | Genera un array vacio del mismo tipo que **x_cpu** y se lo asigna a **r_cpu**.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
**GPU**  |  __syncthreads()       | Sincroniza los hilos de los distintos bloques para que puedan realizar la tarea de ordenamiento correctamente.
CPU      |  SourceModule()        | Define el código del kernel. 
CPU      |  module.get_function() | Genera la función del kernel GPU.
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.

## Conclusión sobre QuickSort

Al ejecutar el algoritmo en CPU y GPU, midiendo los tiempos sobre la ejecución en GPU, la implementación realizada con el procesador GPU da una clara ventaja utilizando su paralelismo y logra tiempos muy cortos.

Sin embargo, investigando y leyendo, se llega a la conclusión que este tipo de algoritmos de ordenamiento no saca todo el potencial de GPU, en este ejemplo el transpaso de vectores de CPU a GPU al ser recursivo demora mucho tiempo, mucho más que la ejecución directamente de todo el algoritmo sobre CPU.

# Referencias leídas

La investigación para la resolución de los problemas la obtuve del material de clase, documentación de pycuda y diversas webs en la que se trataban ejemplos y comparaciones entre ejecuciones cpu y gpu.

https://developer.download.nvidia.com/books/cuda-by-example/cuda-by-example-sample.pdf

https://documen.tician.de/pycuda/

https://www.nvidia.com/content/GTC/documents/1400_GTC09.pdf